# Calculating the mean-free path using conductance calculations

In order to generate the data that is used to fit, run the code in [generate-data.ipynb](generate-data.ipynb). Or download the data, see [README.md](README.md).

In [ ]:
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 

pd.set_option('display.max_columns', None)

# Mean free path $g \sim N_{ch} / (1 + L / \lambda_{MFP})$

We fit:
$1/g \sim  (1/N_{ch} + L / (\lambda_{MFP}N_{ch}))$

In [ ]:
df = pd.read_hdf('data/mean_free_path.hdf')
gb = df.groupby(['disorder', 'L', 'mu', 'num_propagating'])['transmission']
conduction = gb.agg({'means' : np.mean, 'vars' : np.var}).reset_index()

d = {}
for key, gr in conduction.groupby(['L', 'mu']):
    d[key] = (hv.Curve((gr.disorder, gr.means), kdims=['disorder'], vdims=['resistance']) *
              hv.Spread((gr.disorder, gr.means, gr.vars)) *
              hv.Scatter((gr.disorder, gr.means)))[:, 0:25]
hv.util.Dynamic(hv.HoloMap(d, kdims=['L', 'mu']))

### Using a simple polyfit to find  $\lambda_{MFP}$

In [ ]:
for (mu, disorder), gr in conduction.groupby(['mu', 'disorder']):
    modes = gr.num_propagating.values[0]
    fit = np.polyfit(gr.L, 1/gr.means, deg=1)
    mfp = 1 / (modes * fit[0])
    print("mu: {} meV, disorder: {} meV, mfp: {:.0f} nm, num modes: {}".format(mu, disorder, mfp, modes))

### Using bootstrapping to find $\lambda_{MFP}$

In [ ]:
from functools import partial
from scipy import optimize

def fit_bootstrap(p0, datax, datay, function, yerr_systematic=0.0):
    """From http://stackoverflow.com/a/21844726/3447047"""
    errfunc = lambda p, x, y: function(x, p) - y

    # Fit first time
    pfit, perr = optimize.leastsq(errfunc, p0, args=(datax, datay), full_output=0)


    # Get the stdev of the residuals
    residuals = errfunc(pfit, datax, datay)
    sigma_res = np.std(residuals)

    sigma_err_total = np.sqrt(sigma_res**2 + yerr_systematic**2)

    # 100 random data sets are generated and fitted
    ps = []
    for i in range(100):

        randomDelta = np.random.normal(0., sigma_err_total, len(datay))
        randomdataY = datay + randomDelta

        randomfit, randomcov = optimize.leastsq(
            errfunc, p0, args=(datax, randomdataY), full_output=0)

        ps.append(randomfit) 

    ps = np.array(ps)
    mean_pfit = np.mean(ps,0)

    # You can choose the confidence interval that you want for your
    # parameter estimates: 
    Nsigma = 2. # 1sigma gets approximately the same as methods above
                # 1sigma corresponds to 68.3% confidence interval
                # 2sigma corresponds to 95.44% confidence interval
    err_pfit = Nsigma * np.std(ps, 0) 

    pfit_bootstrap = mean_pfit
    perr_bootstrap = err_pfit
    return pfit_bootstrap, perr_bootstrap 

def inv_conductance(L, lambda_mfp, N_ch):
    return 1 / N_ch + L / (lambda_mfp * N_ch)

for (mu, disorder), gr in conduction.groupby(['mu', 'disorder']):
    modes = gr.num_propagating.values[0]
    ff = partial(inv_conductance, N_ch=modes)
    xdata = gr.L
    ydata = 1 / gr.means
    pstart = 1000
    pfit, perr = fit_bootstrap(pstart, xdata, ydata, ff)
    print("mu: {} meV, disorder: {} meV, mfp: {:.0f} nm ± {:.0f}".format(mu, disorder, pfit[0], perr[0]))